In [108]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import copy, deepcopy
import itertools
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [30]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt install msttcorefonts -qq
!rm ~/.cache/matplotlib -rf

# Main Code Starts Here

## Read Data

In [32]:
data_path = "/content/drive/My Drive/Colab Notebooks/Behavioural Study - Indicative Tests/data"
recs2015 = data_path + "/recs2015_public_v4.csv"

data_2015 = pd.read_csv(recs2015, skiprows=1, header=None)
data_2015.sort_values(by=data_2015.columns[0], inplace=True, ignore_index=True)

## Setup

In [33]:
def extract_ESQ_KWH(complete_df, ESQ_idx, KWH_idx, ctrl_idx=None, ctrl_val=None):
    if ctrl_idx is None or ctrl_val is None:
        new_df = complete_df.iloc[:, [ESQ_idx,KWH_idx]].copy()
    elif not isinstance(ctrl_idx, list):
        temp_df = complete_df.loc[complete_df[ctrl_idx]==ctrl_val].copy()
        new_df = temp_df.iloc[:, [ESQ_idx,KWH_idx]].copy()
    else:
        temp_df = deepcopy(complete_df)
        for i in range(len(ctrl_idx)):
            temp_df = temp_df.loc[temp_df[ctrl_idx[i]]==ctrl_val[i]].copy()
        new_df = temp_df.iloc[:, [ESQ_idx,KWH_idx]].copy()
    new_df.rename(columns={ESQ_idx:'ESQ', KWH_idx:'KWH'}, inplace=True)
    new_df.drop(new_df.index[new_df['ESQ'] == -2], inplace=True)
    new_df.drop(new_df.index[new_df['ESQ'] == -8], inplace=True)
    new_df.drop(new_df.index[new_df['ESQ'] == -9], inplace=True)
    new_df = new_df[new_df.notna()].reset_index(drop=True)
    return new_df

In [36]:
def make_controls(indices, values):
    if not (isinstance(indices, list) or isinstance(values, list)):
        raise Exception('Input Error - Inputs must be lists while making controls!')
    if not len(indices)==len(values):
        raise Exception('Input Error - Length mismatch while making controls!')
    for value in values:
        if not isinstance(value, list):
            raise Exception('Input Error - Each set of values must also be a list while making controls!')
    ctrls_val = list(itertools.product(*values))
    ctrls_idx = [indices]*len(ctrls_vals)
    return ctrls_idx, ctrls_val

Initialize Variables Here!

In [89]:
indices = [21, 37, 373, 394, 396]
possible_values = [list(range(1,4)), list(range(1,6)), [1, 0, -2], list(range(2,7)), [0,1]]
ctrls_idx, ctrls_val = make_controls(indices, possible_values)

min_households = 30

In [100]:
def xy_for_LR(complete_df, ESQ_idx, KWH_idx, ctrls_idx=None):
    temp_df = deepcopy(complete_df)
    temp_df.drop(temp_df.index[temp_df[ESQ_idx] == -2], inplace=True)
    temp_df.drop(temp_df.index[temp_df[ESQ_idx] == -8], inplace=True)
    temp_df.drop(temp_df.index[temp_df[ESQ_idx] == -9], inplace=True)
    temp_df = temp_df[temp_df.notna()].reset_index(drop=True)
    if ctrls_idx is not None:
        x = temp_df.loc[:,[ESQ_idx, *ctrls_idx]]
    else:
        x = np.array(temp_df[ESQ_idx]).reshape(-1, 1)
    y = temp_df[KWH_idx]
    return x,y

## Clothes Washer

In [75]:
for i in range(len(ctrls_idx)):
    clothes_washer = extract_ESQ_KWH(data_2015, 354, 600, ctrl_idx=ctrls_idx[i], ctrl_val=ctrls_val[i])
    if len(clothes_washer.index)<min_households:
        continue
    #print(clothes_washer.tail(n=2))
    #clothes_washer.plot(x='KWH', y='ESQ', kind='scatter')
    print('*******************************************')
    print('For ctrl_idx = ', ctrls_idx[i], ' and ctrl_val = ', ctrls_val[i])
    print('Total number of households considered in this set =', len(clothes_washer.index))
    print('Correlation between ESQ and KWH = ', clothes_washer.corr(method='pearson')['ESQ']['KWH'])

*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 4, -2, 2, 0)
Total number of households considered in this set = 35
Correlation between ESQ and KWH =  -0.23189261196176542
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 2, 0)
Total number of households considered in this set = 62
Correlation between ESQ and KWH =  0.043225552918348537
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 3, 0)
Total number of households considered in this set = 52
Correlation between ESQ and KWH =  -0.24228649120007031
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 4, 0)
Total number of households considered in this set = 32
Correlation between ESQ and KWH =  -0.15237803752807003


In [120]:
x,y = xy_for_LR(data_2015, 354, 600, ctrls_idx=indices)
# creating an object of LinearRegression class
LR = LinearRegression()
# fitting the training data
LR.fit(x,y)
#print(LR.intercept_, LR.coef_)


# Manually calculating LR
N = len(x)
p = len(x.columns) + 1  # plus one because LinearRegression adds an intercept term

X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
X_with_intercept[:, 0] = 1
X_with_intercept[:, 1:p] = x.values

beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y.values
print(beta_hat)

# Manually computing Std Error
y_hat = LR.predict(x)
residuals = y.values - y_hat
residual_sum_of_squares = residuals.T @ residuals
sigma_squared_hat = residual_sum_of_squares / (N - p)
var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat
for p_ in range(p):
    standard_error = var_beta_hat[p_, p_] ** 0.5
    print(f"SE(beta_hat[{p_}]): {standard_error}")

[ 35.76118067 -11.88119086   6.8237605    2.29126768   0.2444277
   1.66832102   4.92066538]
SE(beta_hat[0]): 4.028529887100795
SE(beta_hat[1]): 1.5148333920649897
SE(beta_hat[2]): 1.8299012680507443
SE(beta_hat[3]): 0.3820767259955925
SE(beta_hat[4]): 0.20666348304990012
SE(beta_hat[5]): 0.372767035356219
SE(beta_hat[6]): 3.512579140513584


In [128]:
cloth_washer_df = x.copy()
cloth_washer_df['Output'] = y
cloth_washer_df.to_csv(path_or_buf=data_path+'/cloth_washer.csv')

## Dish Washer

In [76]:
for i in range(len(ctrls_idx)):
    dish_washer = extract_ESQ_KWH(data_2015, 355, 602, ctrl_idx=ctrls_idx[i], ctrl_val=ctrls_val[i])
    if len(dish_washer.index)<min_households:
        continue
    #print(dish_washer.tail(n=2))
    #dish_washer.plot(x='KWH', y='ESQ', kind='scatter')
    print('*******************************************')
    print('For ctrl_idx = ', ctrls_idx[i], ' and ctrl_val = ', ctrls_val[i])
    print('Total number of households considered in this set =', len(dish_washer.index))
    print('Correlation between ESQ and KWH = ', dish_washer.corr(method='pearson')['ESQ']['KWH'])

*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 2, 0)
Total number of households considered in this set = 35
Correlation between ESQ and KWH =  0.3426871787574956
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 3, 0)
Total number of households considered in this set = 37
Correlation between ESQ and KWH =  0.011008180469560883


In [129]:
x,y = xy_for_LR(data_2015, 355, 602, ctrls_idx=indices)
# creating an object of LinearRegression class
LR = LinearRegression()
# fitting the training data
LR.fit(x,y)
#print(LR.coef_, LR.intercept_)

# Manually calculating LR
N = len(x)
p = len(x.columns) + 1  # plus one because LinearRegression adds an intercept term

X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
X_with_intercept[:, 0] = 1
X_with_intercept[:, 1:p] = x.values

beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y.values
print(beta_hat)

# Manually computing Std Error
y_hat = LR.predict(x)
residuals = y.values - y_hat
residual_sum_of_squares = residuals.T @ residuals
sigma_squared_hat = residual_sum_of_squares / (N - p)
var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat
for p_ in range(p):
    standard_error = var_beta_hat[p_, p_] ** 0.5
    print(f"SE(beta_hat[{p_}]): {standard_error}")

[-6.48985228 19.06695292 15.51628193  7.05996636 -0.41823797  5.02524472
 13.50565161]
SE(beta_hat[0]): 7.770321323682225
SE(beta_hat[1]): 2.84799931587286
SE(beta_hat[2]): 3.477445433482154
SE(beta_hat[3]): 0.6841207504967161
SE(beta_hat[4]): 0.381965744338795
SE(beta_hat[5]): 0.6810552288646949
SE(beta_hat[6]): 7.329614675722358


In [130]:
dish_washer_df = x.copy()
dish_washer_df['Output'] = y
dish_washer_df.to_csv(path_or_buf=data_path+'/dish_washer.csv')

## Clothes Dryer

In [77]:
for i in range(len(ctrls_idx)):
    clothes_dryer = extract_ESQ_KWH(data_2015, 356, 601, ctrl_idx=ctrls_idx[i], ctrl_val=ctrls_val[i])
    if len(clothes_dryer.index)<min_households:
        continue
    #print(clothes_dryer.tail(n=2))
    #clothes_dryer.plot(x='KWH', y='ESQ', kind='scatter')
    print('*******************************************')
    print('For ctrl_idx = ', ctrls_idx[i], ' and ctrl_val = ', ctrls_val[i])
    print('Total number of households considered in this set =', len(clothes_dryer.index))
    print('Correlation between ESQ and KWH = ', clothes_dryer.corr(method='pearson')['ESQ']['KWH'])

*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 4, -2, 2, 0)
Total number of households considered in this set = 35
Correlation between ESQ and KWH =  -0.004035919269078141
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 2, 0)
Total number of households considered in this set = 62
Correlation between ESQ and KWH =  0.0968242697261541
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 3, 0)
Total number of households considered in this set = 50
Correlation between ESQ and KWH =  0.20087703236815774
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 4, 0)
Total number of households considered in this set = 33
Correlation between ESQ and KWH =  0.135707141108502


In [131]:
x,y = xy_for_LR(data_2015, 356, 601, ctrls_idx=indices)
# creating an object of LinearRegression class
LR = LinearRegression()
# fitting the training data
LR.fit(x,y)
#print(LR.coef_, LR.intercept_)

# Manually calculating LR
N = len(x)
p = len(x.columns) + 1  # plus one because LinearRegression adds an intercept term

X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
X_with_intercept[:, 0] = 1
X_with_intercept[:, 1:p] = x.values

beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y.values
print(beta_hat)

# Manually computing Std Error
y_hat = LR.predict(x)
residuals = y.values - y_hat
residual_sum_of_squares = residuals.T @ residuals
sigma_squared_hat = residual_sum_of_squares / (N - p)
var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat
for p_ in range(p):
    standard_error = var_beta_hat[p_, p_] ** 0.5
    print(f"SE(beta_hat[{p_}]): {standard_error}")

[171.06192988  98.20518162 135.79161974  27.49912338  -1.3528972
   9.0180232   52.82489954]
SE(beta_hat[0]): 54.06119970116715
SE(beta_hat[1]): 20.190915502793693
SE(beta_hat[2]): 24.623571299058298
SE(beta_hat[3]): 5.121670123955898
SE(beta_hat[4]): 2.7773789399924493
SE(beta_hat[5]): 5.000055535340357
SE(beta_hat[6]): 48.034559592634984


In [132]:
cloth_dryer_df = x.copy()
cloth_dryer_df['Output'] = y
cloth_dryer_df.to_csv(path_or_buf=data_path+'/cloth_dryer.csv')

## Freezer

In [78]:
for i in range(len(ctrls_idx)):
    freezer = extract_ESQ_KWH(data_2015, 357, 597, ctrl_idx=ctrls_idx[i], ctrl_val=ctrls_val[i])
    if len(freezer.index)<min_households:
        continue
    #print(freezer.tail(n=2))
    #freezer.plot(x='KWH', y='ESQ', kind='scatter')
    print('*******************************************')
    print('For ctrl_idx = ', ctrls_idx[i], ' and ctrl_val = ', ctrls_val[i])
    print('Total number of households considered in this set =', len(freezer.index))
    print('Correlation between ESQ and KWH = ', freezer.corr(method='pearson')['ESQ']['KWH'])

In [133]:
x,y = xy_for_LR(data_2015, 357, 597, ctrls_idx=indices)
# creating an object of LinearRegression class
LR = LinearRegression()
# fitting the training data
LR.fit(x,y)
#print(LR.coef_, LR.intercept_)

# Manually calculating LR
N = len(x)
p = len(x.columns) + 1  # plus one because LinearRegression adds an intercept term

X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
X_with_intercept[:, 0] = 1
X_with_intercept[:, 1:p] = x.values

beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y.values
print(beta_hat)

# Manually computing Std Error
y_hat = LR.predict(x)
residuals = y.values - y_hat
residual_sum_of_squares = residuals.T @ residuals
sigma_squared_hat = residual_sum_of_squares / (N - p)
var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat
for p_ in range(p):
    standard_error = var_beta_hat[p_, p_] ** 0.5
    print(f"SE(beta_hat[{p_}]): {standard_error}")

[ 5.36958612e+02 -2.48974370e+00 -5.78700146e+01  1.39449318e+01
 -5.23242555e-01 -5.44836579e+00 -7.94193384e+01]
SE(beta_hat[0]): 38.31830568166052
SE(beta_hat[1]): 14.78739879272111
SE(beta_hat[2]): 18.396675507691846
SE(beta_hat[3]): 3.6731948048595036
SE(beta_hat[4]): 2.0561068462913807
SE(beta_hat[5]): 3.718494540315758
SE(beta_hat[6]): 33.032620797067736


In [134]:
freezer_df = x.copy()
freezer_df['Output'] = y
freezer_df.to_csv(path_or_buf=data_path+'/freezer.csv')

## Fridge

In [79]:
for i in range(len(ctrls_idx)):
    fridge = extract_ESQ_KWH(data_2015, 358, 594, ctrl_idx=ctrls_idx[i], ctrl_val=ctrls_val[i])
    if len(fridge.index)<min_households:
        continue
    #print(fridge.tail(n=2))
    #fridge.plot(x='KWH', y='ESQ', kind='scatter')
    print('*******************************************')
    print('For ctrl_idx = ', ctrls_idx[i], ' and ctrl_val = ', ctrls_val[i])
    print('Total number of households considered in this set =', len(fridge.index))
    print('Correlation between ESQ and KWH = ', fridge.corr(method='pearson')['ESQ']['KWH'])

*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 4, -2, 2, 0)
Total number of households considered in this set = 42
Correlation between ESQ and KWH =  -0.1758342413957483
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 2, 0)
Total number of households considered in this set = 62
Correlation between ESQ and KWH =  0.0814505873837689
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 3, 0)
Total number of households considered in this set = 52
Correlation between ESQ and KWH =  0.10598334054945488
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 4, 0)
Total number of households considered in this set = 35
Correlation between ESQ and KWH =  -0.18827922266660801
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and

In [135]:
x,y = xy_for_LR(data_2015, 358, 594, ctrls_idx=indices)
# creating an object of LinearRegression class
LR = LinearRegression()
# fitting the training data
LR.fit(x,y)
#print(LR.coef_, LR.intercept_)

# Manually calculating LR
N = len(x)
p = len(x.columns) + 1  # plus one because LinearRegression adds an intercept term

X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
X_with_intercept[:, 0] = 1
X_with_intercept[:, 1:p] = x.values

beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y.values
print(beta_hat)

# Manually computing Std Error
y_hat = LR.predict(x)
residuals = y.values - y_hat
residual_sum_of_squares = residuals.T @ residuals
sigma_squared_hat = residual_sum_of_squares / (N - p)
var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat
for p_ in range(p):
    standard_error = var_beta_hat[p_, p_] ** 0.5
    print(f"SE(beta_hat[{p_}]): {standard_error}")

[ 545.80918681 -117.86714051 -127.13305296   65.16905744    0.83874057
   21.01339552  -39.51424818]
SE(beta_hat[0]): 36.39040392213757
SE(beta_hat[1]): 13.482760502483531
SE(beta_hat[2]): 15.529714837973676
SE(beta_hat[3]): 3.277315571345208
SE(beta_hat[4]): 1.806290290388724
SE(beta_hat[5]): 3.2818504619747424
SE(beta_hat[6]): 29.174185010992204


In [136]:
fridge_df = x.copy()
fridge_df['Output'] = y
fridge_df.to_csv(path_or_buf=data_path+'/fridge.csv')

## Light Bulbs

In [80]:
for i in range(len(ctrls_idx)):
    light_bulbs = extract_ESQ_KWH(data_2015, 359, 603, ctrl_idx=ctrls_idx[i], ctrl_val=ctrls_val[i])
    if len(light_bulbs.index)<min_households:
        continue
    #print(light_bulbs.tail(n=2))
    #light_bulbs.plot(x='KWH', y='ESQ', kind='scatter')
    print('*******************************************')
    print('For ctrl_idx = ', ctrls_idx[i], ' and ctrl_val = ', ctrls_val[i])
    print('Total number of households considered in this set =', len(light_bulbs.index))
    print('Correlation between ESQ and KWH = ', light_bulbs.corr(method='pearson')['ESQ']['KWH'])

*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 4, -2, 2, 0)
Total number of households considered in this set = 42
Correlation between ESQ and KWH =  -0.11193767030631553
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 2, 0)
Total number of households considered in this set = 62
Correlation between ESQ and KWH =  -0.16810926980355742
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 3, 0)
Total number of households considered in this set = 52
Correlation between ESQ and KWH =  -0.12777926085549793
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 4, 0)
Total number of households considered in this set = 36
Correlation between ESQ and KWH =  0.2628056111892147
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  a

In [137]:
x,y = xy_for_LR(data_2015, 359, 603, ctrls_idx=indices)
# creating an object of LinearRegression class
LR = LinearRegression()
# fitting the training data
LR.fit(x,y)
#print(LR.coef_, LR.intercept_)

# Manually calculating LR
N = len(x)
p = len(x.columns) + 1  # plus one because LinearRegression adds an intercept term

X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
X_with_intercept[:, 0] = 1
X_with_intercept[:, 1:p] = x.values

beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y.values
print(beta_hat)

# Manually computing Std Error
y_hat = LR.predict(x)
residuals = y.values - y_hat
residual_sum_of_squares = residuals.T @ residuals
sigma_squared_hat = residual_sum_of_squares / (N - p)
var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat
for p_ in range(p):
    standard_error = var_beta_hat[p_, p_] ** 0.5
    print(f"SE(beta_hat[{p_}]): {standard_error}")

[-188.93944163 -241.17922582   52.13285945  156.42722181   -4.55798736
  107.44202947 -108.51195459]
SE(beta_hat[0]): 92.74822392804731
SE(beta_hat[1]): 33.94851898881593
SE(beta_hat[2]): 39.899887184888144
SE(beta_hat[3]): 8.418399653060643
SE(beta_hat[4]): 4.65490212296478
SE(beta_hat[5]): 8.441206582753617
SE(beta_hat[6]): 74.78485892327666


In [138]:
light_bulbs_df = x.copy()
light_bulbs_df['Output'] = y
light_bulbs_df.to_csv(path_or_buf=data_path+'/light_bulbs.csv')

## Water Heater

In [81]:
for i in range(len(ctrls_idx)):
    water_heater = extract_ESQ_KWH(data_2015, 360, 593, ctrl_idx=ctrls_idx[i], ctrl_val=ctrls_val[i])
    if len(water_heater.index)<min_households:
        continue
    #print(water_heater.tail(n=2))
    #water_heater.plot(x='KWH', y='ESQ', kind='scatter')
    print('*******************************************')
    print('For ctrl_idx = ', ctrls_idx[i], ' and ctrl_val = ', ctrls_val[i])
    print('Total number of households considered in this set =', len(water_heater.index))
    print('Correlation between ESQ and KWH = ', water_heater.corr(method='pearson')['ESQ']['KWH'])

*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 4, -2, 2, 0)
Total number of households considered in this set = 42
Correlation between ESQ and KWH =  -0.06200840831266406
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 2, 0)
Total number of households considered in this set = 63
Correlation between ESQ and KWH =  -0.1818337862628327
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 3, 0)
Total number of households considered in this set = 52
Correlation between ESQ and KWH =  -0.043076508463660186
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  and ctrl_val =  (1, 5, -2, 4, 0)
Total number of households considered in this set = 36
Correlation between ESQ and KWH =  0.08488410349283391
*******************************************
For ctrl_idx =  [21, 37, 373, 394, 396]  

In [139]:
x,y = xy_for_LR(data_2015, 360, 593, ctrls_idx=indices)
# creating an object of LinearRegression class
LR = LinearRegression()
# fitting the training data
LR.fit(x,y)
#print(LR.coef_, LR.intercept_)

# Manually calculating LR
N = len(x)
p = len(x.columns) + 1  # plus one because LinearRegression adds an intercept term

X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
X_with_intercept[:, 0] = 1
X_with_intercept[:, 1:p] = x.values

beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y.values
print(beta_hat)

# Manually computing Std Error
y_hat = LR.predict(x)
residuals = y.values - y_hat
residual_sum_of_squares = residuals.T @ residuals
sigma_squared_hat = residual_sum_of_squares / (N - p)
var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat
for p_ in range(p):
    standard_error = var_beta_hat[p_, p_] ** 0.5
    print(f"SE(beta_hat[{p_}]): {standard_error}")

[1247.58367996  186.86118337  219.02730626   47.05335271   25.52243106
  -78.16157756 -443.08799504]
SE(beta_hat[0]): 168.9455334420096
SE(beta_hat[1]): 64.31095086864961
SE(beta_hat[2]): 73.15129263717787
SE(beta_hat[3]): 15.4199083513584
SE(beta_hat[4]): 8.506430644775092
SE(beta_hat[5]): 15.333412483172536
SE(beta_hat[6]): 136.5492608247855


In [140]:
water_heater_df = x.copy()
water_heater_df['Output'] = y
water_heater_df.to_csv(path_or_buf=data_path+'/water_heater.csv')